### Libraries

In [1]:
import sys
sys.path.insert(0,'/Users/jarad/Fake Folder/Python Libraries')

from jb_libraries import *
%matplotlib inline

### Script settings

In [2]:
csv_path = '/Users/jarad/Fake Folder/Fab/Recurring/Time to Test and Prep Fab Parts/CSVs/'
close_workbook = 'yes'

In [3]:
n = 90
now = dt.datetime.now().date()
x_days_ago = (now - pd.DateOffset(days = n)).date()

date_start = str(x_days_ago)
date_end = str(now)

print('Data is from {} to {}, which is {} days'.format(date_start,date_end,n))

Data is from 2018-12-18 to 2019-03-18, which is 90 days


In [4]:
workbook_title = 'Previous 90 days ending {} - Time to Test and Prep Fab Parts'.format(date_end)

### Get data

In [5]:
processes_main = pd.read_sql(
'''
SELECT
process_id,
wo_id,
process_group
FROM processes
WHERE DATE(time_start) BETWEEN ' '''+ date_start +''' ' AND ' '''+ date_end +''' ' 
AND wo_id != 0 # to exclude reworks
AND process_group IN ('test','prep','secondary') # get only these groups
AND time_end != '0000-00-00 00:00:00' # to exclude open processes
''', db)

col_fix(processes_main)

In [6]:
p = list(set(processes_main['process id']))

sp_actions_main = pd.read_sql(
'''
SELECT
process_id,
spa_qty,
spa_time_start,
spa_time_end
FROM sp_actions
WHERE process_id IN '''+ str(tuple(p)) +'''
''', db)

col_fix(sp_actions_main)

e = sp_actions_main['spa time end'] - sp_actions_main['spa time start']
minutes = (e.dt.days * 24 * 60 * 60 + e.dt.seconds)/60
sp_actions_main['time elapsed in minutes'] = minutes

In [7]:
skus_main = pd.read_sql(
'''
SELECT
sku_id,
k.part_id,
pd.products_name
FROM skus k
LEFT JOIN products_description pd ON k.part_id = pd.part_id
''', db)

col_fix(skus_main)

In [8]:
w = list(set(processes_main['wo id']))

work_orders_main = pd.read_sql(
'''
SELECT
wo_id,
sku_id,
wo_qty AS work_order_qty
FROM work_orders 
WHERE wo_id IN '''+ str(tuple(w)) +'''
AND wo_status = 'completed' # only include completed WOs
''', db)

col_fix(work_orders_main)

for c in ['part id','products name']:
    work_orders_main[c] = work_orders_main['sku id'].map(dict(zip(skus_main['sku id'], skus_main[c])))
    
work_orders_main['products name'].fillna('no part name', inplace = True)    

### Structure it

In [9]:
main = sp_actions_main.copy()

# map from processes_main
cols = processes_main.columns.tolist()
cols.remove('process id')
for c in cols:
    main[c] = main['process id'].map(dict(zip(processes_main['process id'],processes_main[c])))
    
# map from work_orders_main    
cols = work_orders_main.columns.tolist()
cols.remove('wo id')
for c in cols:
    main[c] = main['wo id'].map(dict(zip(work_orders_main['wo id'],work_orders_main[c])))
    
# check nulls    
n = main[main.isnull().any(1)]
if n.empty == True: # if there are no nulls, move on
    pass
else: # if there are nulls, check the wo_status of these wo_ids; if the statuses are anything but "completed", move on
    
    n_ls = list(set(n['wo id']))

    check = pd.read_sql(
    '''
    SELECT
    wo_status,
    COUNT(*) AS count
    FROM work_orders
    WHERE wo_id IN '''+ str(tuple(n_ls)) +'''
    ''', db)

    col_fix(check)    

    count = 0
    for index, row in check.iterrows():
        count += row['count']
        if row['wo status'] == 'completed':
            raise ValueError('check your nulls')

            if count != len(n_ls): # if the count of nulls from your df matches that in the db, move on
                raise ValueError('check your nulls')

main.dropna(inplace = True)
main.reset_index(drop = True, inplace = True)

### Get time elapsed

In [10]:
time = main.groupby('part id')[['time elapsed in minutes']].sum()

### Get WO qty

In [11]:
wo_qty = work_orders_main.groupby('part id')[['work order qty']].sum()

### Get Wo Ids

In [12]:
a1 = work_orders_main[['part id','wo id']].drop_duplicates()
a2 = a1.groupby('part id')['wo id'].apply(lambda x: ', '.join(x.map(str)))
wo_ids = pd.DataFrame(a2)
wo_ids.columns = ['wo ids']

### Data by part

In [13]:
by_part = time.join(wo_qty).join(wo_ids)
by_part.reset_index(inplace = True)
by_part['part id'] = by_part['part id'].map(int)

by_part['products name'] = by_part['part id'].map(dict(zip(main['part id'], main['products name'])))
    
by_part['min per'] = by_part['time elapsed in minutes']/by_part['work order qty']

l1 = 'https://volcano.adafruit.com/volcano/ada_plm.php?/Parts/'
l2 = '/PartProfile/'

by_part['link'] = l1 + by_part['part id'].map(str) + l2

### Rearrange columns and sort

In [14]:
new_cols = ['part id',
            'products name',
            'wo ids',
            'time elapsed in minutes',
            'work order qty',
            'min per',
            'link']

old_cols = by_part.columns.tolist()

s1 = set(old_cols)
s2 = set(new_cols)
s3 = s1.symmetric_difference(s2)

if len(s3) != 0:
    raise ValueError('check ur columns')
else:
    by_part = by_part[new_cols]
    
by_part.sort_values('time elapsed in minutes', ascending = False, inplace = True)    

In [15]:
by_part.head()

,part id,products name,wo ids,time elapsed in minutes,work order qty,min per,link
80,3093,Adafruit CRICKIT for Circuit Playground Express,7724,5763.950000,856,6.733586,https://volcano.adafruit.com/volcano/ada_plm.php?/Parts/3093/PartProfile/
54,2598,Adafruit Feather M0 WiFi - ATSAMD21 + ATWINC1500 (accounts only),"7997, 8182",4205.583333,1640,2.564380,https://volcano.adafruit.com/volcano/ada_plm.php?/Parts/2598/PartProfile/
117,3957,Adafruit CRICKIT HAT for Raspberry Pi,"7685, 7927",3042.583333,827,3.679061,https://volcano.adafruit.com/volcano/ada_plm.php?/Parts/3957/PartProfile/
103,3500,Adafruit Trinket M0 - for use with CircuitPython & Arduino IDE,7872,2263.116667,2400,0.942965,https://volcano.adafruit.com/volcano/ada_plm.php?/Parts/3500/PartProfile/
27,1752,Stereo 20W Class D Audio Amplifier - MAX9744,7925,1452.100000,1302,1.115284,https://volcano.adafruit.com/volcano/ada_plm.php?/Parts/1752/PartProfile/


### Find outliers by time elapsed

In [16]:
for_outliers = main.groupby('process group').agg({'time elapsed in minutes':['mean','std']})
for_outliers.columns = for_outliers.columns.droplevel(0)
for_outliers['upper'] = for_outliers['mean'] + 3 * for_outliers['std']
for_outliers.reset_index(inplace = True)

def find_outlier(df):
    
    group = df['process group']
    time = df['time elapsed in minutes']
    upper = for_outliers[for_outliers['process group'] == group]['upper'].item()
    
    if time > upper:
        return 'yes'
    else:
        return 'no'
    
main['outlier'] = main.apply(find_outlier, axis = 1)

In [17]:
print('outliers below')
main[main['outlier'] == 'yes'].sort_values(['process group','time elapsed in minutes'], ascending = [False,False])

outliers below


,process id,spa qty,spa time start,spa time end,time elapsed in minutes,wo id,process group,sku id,work order qty,part id,products name,outlier
657,166034,163,2019-02-19 08:40:16,2019-02-19 14:30:20,350.066667,7927,test,12651.0,470.0,3957.0,Adafruit CRICKIT HAT for Raspberry Pi,yes
626,166009,624,2019-01-29 08:41:02,2019-01-29 14:18:08,337.100000,7925,test,8583.0,1302.0,1752.0,Stereo 20W Class D Audio Amplifier - MAX9744,yes
251,160030,410,2019-01-23 09:57:07,2019-01-23 15:32:53,335.766667,7724,test,12369.0,856.0,3093.0,Adafruit CRICKIT for Circuit Playground Express,yes
705,167005,206,2019-01-18 08:51:12,2019-01-18 14:03:01,311.816667,7958,test,12575.0,498.0,3938.0,Adafruit NeoTrellis M4 Mainboard - featuring SAMD51,yes
34,145208,114,2019-01-24 09:44:12,2019-01-24 14:22:55,278.716667,7170,test,8693.0,167.0,2219.0,TFP401 HDMI/DVI Decoder to 40-Pin TTL Breakout - With Touch,yes
415,162907,30,2019-02-28 10:51:42,2019-02-28 15:02:12,250.500000,7828,test,9891.0,190.0,3531.0,Adafruit 128x64 OLED Bonnet for Raspberry Pi,yes
835,168797,221,2019-01-11 08:00:20,2019-01-11 12:04:54,244.566667,8026,test,13029.0,300.0,1059.0,Flora Wearable Ultimate GPS Module,yes
540,164281,190,2019-01-08 09:06:48,2019-01-08 13:01:12,234.400000,7872,test,11110.0,2400.0,3500.0,Adafruit Trinket M0 - for use with CircuitPython & Arduino IDE,yes
349,161934,170,2018-12-24 09:06:47,2018-12-24 12:59:47,233.000000,7799,test,12920.0,449.0,4026.0,Adafruit STEMMA Soil Sensor - I2C Capacitive Moisture Sensor,yes
614,165859,90,2019-02-18 13:56:11,2019-02-18 17:47:00,230.816667,7923,test,11588.0,306.0,3787.0,"Adafruit 1.54"" 240x240 Wide Angle TFT LCD Display with MicroSD",yes


### Check it

In [18]:
# check nulls
n = by_part[by_part.isnull().any(1)]
if n.empty == False:
    raise ValueError('check ur nulls')
    
# qty totals    
t1 = work_orders_main['work order qty'].sum()
t2 = by_part['work order qty'].sum()

if t1 - t2 != 0:
    raise ValueError('check ur totals')    
    
# part count
s1 = set(work_orders_main['part id'])
s2 = set(by_part['part id'])

if len(s1) != len(s2):
    raise ValueError('check ur part count')

*****
# Excel
*****

### Set up workbook

In [19]:
workbook = xlsxwriter.Workbook(csv_path + workbook_title + '.xlsx',
                               {'nan_inf_to_errors': True,
                               'strings_to_numbers': True})

tabs = ['Data By Part']

#=========================
# Formats
#=========================

colors = ['#343635',
          '#2e4874',
          '#7eaba4',
          '#928c85',
          '#347c83',
          '#bfb9d6']
colors = colors * 2

title = workbook.add_format({'font_size':25,
                             'font_name':'Arial (Bold)'})

subtitle = workbook.add_format({'font_size':15,
                                'font_name':'Arial (Bold)'})

col_names = workbook.add_format({'font_name':'Arial (Bold)',
                                 'font_color':'white',
                                 'valign':'vcenter',
                                 'align':'center',
                                 'bg_color':colors[2],
                                 'bottom':1,
                                 'top':1,
                                 'left':1,
                                 'right':1})

right_align = workbook.add_format({'align':'right'})

number = workbook.add_format({'num_format':'#,##0'})
number2 = workbook.add_format({'num_format':'#,##0.0'})
dummy = workbook.add_format({'font_color':'black'})

for tab in tabs:
    workbook.add_worksheet(tab) # create each tab

my_worksheets = {}
for sht in workbook.worksheets():
    my_worksheets[sht.get_name()] = sht # create dict like tab_name:worksheet_instance

#=========================
# Apply same formatting to each worksheet
#=========================

for k, v in my_worksheets.items():
    sht = my_worksheets[k]
    sht.write(0, 0,
              k.title(),
              title)
    sht.write(1, 0,
              workbook_title,
              subtitle)

### Write worksheets

In [20]:
sht = my_worksheets['Data By Part']

fmt = {'time elapsed in minutes':number,
      'work order qty':number,
      'min per':number2,
      'wo ids':right_align}

df = by_part.copy()

start_row = 3
start_col = 0

#=========================
# disclaimer 01
#=========================

ls = list(set(main['process group']))
w = '*times are summed over the '
for i in range(len(ls)):
    if i != len(ls) - 1:
        w += ls[i] + ', '
    else:
        w += 'and ' + ls[i] + ' steps, and the data only includes completed work orders.'
        
sht.write(start_row - 1,
         start_col,
         w)

#=========================
# data
#=========================

for i in range(len(df.columns)):
    sht.write(start_row + 1,
              start_col + i,
              df.columns[i].title(),
              col_names)

    len1 = [len(str(x)) for x in df.iloc[:, i]]
    len2 = [len(df.columns[i])]
    len3 = np.max(len1 + len2)

    sht.set_column(start_col + i,
                   start_col + i,
                   len3)

    for j in range(len(df)):
        sht.write(start_row + 2 + j, 
                  start_col + i,
                  df.iloc[j,i],
                  fmt[df.columns[i]] if df.columns[i] in list(fmt.keys()) else dummy)
        
if close_workbook == 'yes':
    workbook.close()        

In [21]:
print('done')

done
